# ***Train custom Face Mask Detector by YOLOv3m***

We will train the custom YOLO detector for Mask detection on the Face and save to the ONNX format for production using.

### ***Check GPU resources***

In [ ]:
!nvidia-smi

### ***Install libraries***

In [ ]:
!pip install ultralytics onnx onnxruntime

### ***Import libraries***

In [ ]:
# Import Libraries
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt

from zipfile import ZipFile
from urllib.request import urlretrieve

from IPython.display import HTML
from matplotlib.animation import FuncAnimation

from IPython.display import YouTubeVideo, display, HTML, Video

%matplotlib inline

## ***Assignment***

# Introduction

## Project content
1. Overview
2. Expected Output
3. Data
4. Task
5. Submission
6. Marks Distribution

## ***1. Overview***

In the last lesson, we have learned how to train a custom Object Detector using YOLO v3 network for a single object. In this project, you will extend that learning by building a 2-Class Object Detector using Yolo v3 as well as the more recent Yolo v4. In the process, you will also learn how to browse other GitHub repositories and find out relevant information to complete your task.

Since Covid-19 has had such a huge impact on our lives, we will build a Mask and No-mask Detector. So, the classes are:

- Class 0 - Face with Mask
- Class 1 - Face without Mask

## ***2. Expected Output***

Given below are the expected outputs of the test images and videos.

<center><img src="https://github.com/RadimKozl/OpenCV_Project4/blob/main/img/test-image1-sol.png?raw=1" alt="Test Image1" style="width: 800px;"/></center>
<center><img src="https://github.com/RadimKozl/OpenCV_Project4/blob/main/img/test-image2-sol.png?raw=1" alt="Test Image2" style="width: 800px;"/></center>
<center><img src="https://github.com/RadimKozl/OpenCV_Project4/blob/main/img/test-image3-sol.png?raw=1" alt="Test Image3" style="width: 800px;"/></center>
<center><img src="https://github.com/RadimKozl/OpenCV_Project4/blob/main/img/test-image4-sol.png?raw=1" alt="Test Image4" style="width: 800px;"/></center>

----------------------------------------------------------

### **Video**

<center><iframe width="420" height="315" src="https://www.youtube.com/watch?v=1982JxViLig"></iframe> </center>

